In [1]:
import requests
import pandas as pd
import numpy as np
import regex as re
from random import randint

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import json_normalize
import matplotlib.pyplot as plt
import pickle
from time import sleep

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
top_songs = pd.read_csv('top_songs.csv')
top_songs

,song,artist
0,Lovin On Me,Jack Harlow
1,Lil Boo Thang,Paul Russell
2,I Remember Everything (feat. Kacey Musgraves),Zach Bryan
3,White Horse,Chris Stapleton
4,Need A Favor,Jelly Roll
...,...,...
145,Smells Like Teen Spirit,Nirvana
146,Like a Rolling Stone,Bob Dylan
147,A Change Is Gonna Come,Sam Cooke
148,Fight the Power,Public Enemy


In [3]:
playlist_df = pd.read_csv('playlist_df.csv')
playlist_df.head()

,track.name,name,cluster
0,2K,Nosaj Thing,3
1,4 Billion Souls,The Doors,1
2,4 Minute Warning,Radiohead,16
3,7 Element,Vitas,2
4,#9 Dream,R.E.M.,9


In [4]:
def open_spotify_connection():
    #Opens a new spotify connection
    secrets_file = open("secrets.txt","r")
    string = secrets_file.read()
    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            #print(line.split(':'))
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))
    return sp;

In [5]:
def remove_special_characters(text):
    # Remove special characters using a regular expression
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

In [8]:
def recommend_song(input_song, input_artist):

    # Convert the user input and song names to lowercase for case-insensitive comparison
    input_song_lower = input_song.lower()
    input_artist_lower = input_artist.lower()

    top_songs_lower = top_songs.apply(lambda x: x.str.lower().apply(remove_special_characters))
    top_songs_lower['song'] = top_songs_lower['song'].apply(remove_special_characters)

    # Check if the user-inputted song is in the top songs list (case-insensitive)
    if input_song_lower in top_songs_lower['song'].values:
        # Filter out the user-inputted song from the DataFrame
        top_songs_without_input = top_songs[
            top_songs_lower['song'] != input_song_lower
        ]

        # Recommend a random song from the filtered df
        recommended_song = top_songs_without_input.sample().squeeze()

        print(f"You might also like: {recommended_song['song']} by {recommended_song['artist']}")
        
    else:
        
        #Opening the spotify connection
        sp = open_spotify_connection()

        # Search for the user-inputted song and artist on Spotify
        search_results = sp.search(q=f"{input_song} {input_artist}", type='track', limit =1 )

        if not search_results['tracks']['items']:
            print("Song not found on Spotify. Please try another song.")
            return

        # Get the track ID for the user-inputted song
        if search_results['tracks']['total']!=0 :
            track_id = search_results['tracks']['items'][0]['id']
            print(track_id)
        
        # Getting the audio features using the id
        new_data = sp.audio_features(track_id)
        print(type(new_data[0]))
        data_df = json_normalize(new_data[0]) #normalizing the data to a df
        num_data_df = data_df.select_dtypes(include='number') #getting the numerical data only to fit
        print(num_data_df)

        #Using pickle to get the existing scaler and kmean
        kmeans = pickle.load(open('kmean.pkl', 'rb'))
        scaler = pickle.load(open('scaler.pkl','rb'))
        X_prep = scaler.transform(num_data_df)
        predicted_cluster = kmeans.predict(X_prep)
        
        # Predict the cluster for the input song using the clustering model
        cluster = predicted_cluster[0]
        #print(cluster)
        
        # Filter songs in the same cluster from the playlist DataFrame and
        # Recommend a random song from the same cluster
        recommended_song = playlist_df[playlist_df['cluster']== cluster].sample().squeeze()
        print(recommended_song)

        print(f"You might also like: {recommended_song['track.name']} by {recommended_song['name']}")

        # Introduce a respectful nap to avoid hitting rate limits
        sleep(randint(1, 3000) / 1000)


In [10]:
# Get user input for song and artist
input_song = input("Enter a song: ")
input_artist = input("Enter the artist: ")
recommend_song(input_song, input_artist )

Enter a song: why can't i be you
Enter the artist: the cure
3RcrFgETR1QGjzV6ZzIRBy
<class 'dict'>
   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.629   0.949    0    -4.664     0        0.048         0.461   

   instrumentalness  liveness  valence    tempo  duration_ms  time_signature  
0          0.000009    0.0522    0.719  107.798       194133               4  
track.name    Eye of the Tiger
name                  Survivor
cluster                      7
Name: 25000, dtype: object
You might also like: Eye of the Tiger by Survivor
